# IBM SPSS Modeler SNA包

> 社交网络分析（SNA）是 Modeler 15 增加的一个新功能。目前有两种算法支持这个功能，分别称作 GA 和 DA。

- GA 全称 Group Analysis, 是一种基于群体的分析方法。
- DA 全称 Diffusion Analysis，着眼于计算一些人的行为对网络中其他人的冲击强度。


1. GA 收到这些数据后，会进行如下的分析：

  - 根据共有邻接节点的数量，量化各个节点（也就是张三、李四、王五。。。）之间联系的强度；
  - 保留高强度的联系，去除低强度的联系。进行此步骤后，社交网络会退化成几个内部联系多、外部联系少的次网络，以及很多孤立的节点。每个次网络对应一个群体（group），称为这个群体的核 (kernel)；
  - 把那些孤立的节点连到距离他们最近的群体去。上一步里暂时去除的低强度的联系，在这一步发挥了主要作用；
  - 对各个群体以及群体里的个体进行分析画像，例如评估群体内每个个体的地位，找出“领袖”，计算群体密度，等等。

2. DA 的算法原理要更简单一些。DA 不会将网络分成群体，而是在原网络上进行计算。DA 着眼于计算一些人的行为对网络中其他人的冲击强度。

  - DA会构筑一个有向加权网络，如下图所示。网络中的节点代表人、组织、计算机或者其他信息或知识处理实体；连线表示节点之间的关系或信息流动；连线的方向表示了关系的主动被动方，或者信息流动的方向（通常为双向，图 7 省略了此内容）。
  - DA 还需要有行为发生的人的名单。这里的行为可以是从公司辞职，更换手机服务商，试用了某种产品，等等。这些人被称作“初始扩散点”(Initial diffusing seeds)，由图 7 中的红色节点表示。接下来，设定初始扩散点的冲击强度，然后采用衰败扩散过程就可以估计出其他节点所受到的冲击大小。
![img](https://www.ibm.com/developerworks/cn/data/library/techarticle/dm-1206zhangxy/Figure7.JPG)

## 参考
1. [实验数据集-邮件](http://snap.stanford.edu/data/email-EuAll.html)
2. [使用IBM SPSS Modeler进行社交网络分析](https://www.ibm.com/developerworks/cn/data/library/techarticle/dm-1206zhangxy/index.html)
3. [精品案例 | 某移动通讯公司客户流失预警分析](http://www.sohu.com/a/131761172_455817)
4. [[机器学习实战]使用 scikit-learn 预测用户流失](https://blog.csdn.net/BaiHuaXiu123/article/details/62063415)

In [17]:
import gzip
import pandas as pd
import re

df = pd.DataFrame(columns = ["from", "to"]) 

## 1. 源数据处理与整理

数据格式

发件人编号 收件人编号

In [ ]:
%%time
idx = 0
with gzip.open(r"data\email-EuAll.txt.gz", "r") as pf:
    for line in pf:
        if not str(line).startswith("b'#"):
            ft = re.findall(r"\d+", str(line))
            df.loc[idx, "from"] = ft[0]
            df.loc[idx, "to"] = ft[1]
            idx += 1
            
df.to_pickle(r"data\email-EuAll.pkl")

In [10]:
df = pd.read_pickle(r"data\email-EuAll.pkl")

False

## 2. 案例一：客户流失预警

> 需要数据
1. Demo_GA_KPI， 关系对 
2. Demo_GA_churner， 已流失客户名单， 随机生成